# 3D Ocean Model Visualization (ROMS)

This example visualizes output from the [Regional Ocean Modeling System (ROMS)](https://www.myroms.org/) — a widely-used ocean circulation model. ROMS uses **curvilinear horizontal coordinates** (2D lat/lon arrays) and **terrain-following vertical coordinates** (sigma levels), making it a perfect test case for pyvista-xarray's `StructuredGrid` support.

We'll create a 3D visualization of ocean salinity, showing how the salt distribution varies with depth across the Texas-Louisiana shelf.

In [ ]:
import numpy as np
import pyvista as pv
import pvxarray  # noqa: F401
import xarray as xr

pv.set_plot_theme("document")

## Load ROMS Data

The xarray tutorial includes a ROMS example dataset from the Texas-Louisiana shelf. It has:
- **2D horizontal coordinates**: `lon_rho(eta_rho, xi_rho)` and `lat_rho(eta_rho, xi_rho)` — a curvilinear grid
- **Terrain-following vertical coordinate**: `s_rho` — sigma levels from bottom (-1) to surface (0)
- **Salinity**: `salt(ocean_time, s_rho, eta_rho, xi_rho)` — the variable we'll visualize

In [ ]:
ds = xr.tutorial.open_dataset("ROMS_example.nc")

# Select one timestep of salinity
salt = ds.salt.isel(ocean_time=0)
print(f"Shape: {salt.shape}")
print(f"Coordinates: {list(salt.coords)}")
print(f"lon_rho shape: {ds.lon_rho.shape} (2D curvilinear)")
print(f"s_rho shape: {ds.s_rho.shape} (1D vertical)")

## Surface Salinity (2D Slice)

First, let's look at the surface layer. Selecting the topmost sigma level gives us a 2D field with curvilinear coordinates. pyvista-xarray will create a `StructuredGrid` since `lon_rho` and `lat_rho` are 2D arrays:

In [ ]:
# Surface layer (topmost sigma level)
surface = salt.isel(s_rho=-1)

mesh = surface.pyvista.mesh(x="lon_rho", y="lat_rho")
print(f"Mesh type: {type(mesh).__name__}")

mesh.plot(cpos="xy", cmap="viridis")

## Vertical Cross-Section

Extract a cross-section along one eta index to see the vertical structure. We use `s_rho` as the Z coordinate, which represents the fraction of the water column from bottom (-1) to surface (0):

In [ ]:
# Cross-section at mid-shelf
cross = salt.isel(eta_rho=100)

mesh = cross.pyvista.mesh(x="lon_rho", y="s_rho")
mesh.plot(cpos="xy", cmap="viridis")

## Compare Two Timesteps

The dataset contains two timesteps (Aug 1 and Aug 8, 2001). Let's compare the surface salinity side-by-side to see how the freshwater plume from the Mississippi River evolves:

In [ ]:
pl = pv.Plotter(shape=(1, 2))

for i, t in enumerate([0, 1]):
    pl.subplot(0, i)
    surface = ds.salt.isel(ocean_time=t, s_rho=-1)
    mesh = surface.pyvista.mesh(x="lon_rho", y="lat_rho")
    pl.add_mesh(mesh, cmap="viridis", clim=(0, 37))
    pl.add_text(str(ds.ocean_time.values[t])[:10], font_size=10)
    pl.view_xy()

pl.link_views()
pl.show()